In [39]:
from os import chdir, path, getcwd
for i in range(10):
    if path.isfile("checkcwd"):
        break
    chdir(path.pardir)
if path.isfile("checkcwd"):
    pass
else:
    raise Exception("Something went wrong. cwd=" + getcwd())

In [40]:
from tqdm.auto import tqdm
from collections import defaultdict
import sys

import requests
import random
from bs4 import BeautifulSoup
from json import loads
from html import unescape
import pandas as pd

In [41]:
RATINGS_DIR = 'resources/ratings'


USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.170 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15'
]


In [42]:
import os
rating_files = os.listdir(RATINGS_DIR)
len(rating_files)

43

In [43]:
rating_filepaths = [os.path.join(RATINGS_DIR, f) for f in rating_files]
uids = defaultdict(int)
fids = defaultdict(int)
for filepath in tqdm(rating_filepaths):
    with open(filepath, 'r') as fr:
        lines = fr.readlines()
    for line in lines:
        splitted_line = line.split(sep=',')
        uids[splitted_line[0]] += 1
        fids[splitted_line[1]] += 1

  0%|          | 0/43 [00:00<?, ?it/s]

In [44]:
sys.getsizeof(fids)

15379208

In [45]:
def get_film_data(id: str):
    film_url = f'https://www.imdb.com/title/tt{id}'
    r = requests.get(
        film_url,
        headers={'User-Agent': random.choice(USER_AGENTS)}
    )
    soup = BeautifulSoup(r.text, 'html.parser').head
    ld_json_script = soup.find('script', type='application/ld+json')
    json_data = loads(ld_json_script.string)
    return json_data

In [46]:
film_id = '0910970'
walle = get_film_data(film_id)

In [47]:
walle['duration']
dt = pd.Timedelta(walle['duration'])
int(dt.total_seconds())

5880

In [52]:
test_fids = list(fids.keys())[:5]
test_fids

['0111161', '0109830', '0110912', '0133093', '0068646']

In [61]:
with open('test.csv', 'w') as fw:
    # fw.write(f'fid,name,description,ratingCount,ratingValue,contentRating,genre,keywords,duration\n')
    for fid in test_fids:
        json_film = get_film_data(fid)
        fw.write(f'"{fid}","{json_film['name']}","{json_film['description']}",{json_film['aggregateRating']['ratingCount']},')
        fw.write(f'{json_film['aggregateRating']['ratingValue']},{json_film['contentRating']},')
        fw.write(f'"{','.join(json_film['genre'])}","{json_film['keywords']}",{int(pd.Timedelta(json_film['duration']).total_seconds())}\n')